# Automated City Pairing for Causal Analysis

This notebook iterates through all available cities, identifies suitable control candidates using advanced metrics, and exports the findings to `city_pairings.csv`. This automates the selection process for any configuration of target and controls.

In [ ]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Constants from causal_impact.ipynb
target_col = 'Barcelona' # Default example
pre_beg, pre_end = '2023-01-01', '2023-05-31'
t1_thresh = 0.8
t2_thresh = 0.6

## 1. Load and Prepare Data

In [ ]:
df_long = pd.read_csv('sales_data.csv')
df_long['Date'] = pd.to_datetime(df_long['Date'])
df = df_long.pivot(index='Date', columns='City', values='Value')
df.index.freq = 'D'

cities = sorted(df.columns.tolist())
print(f"Processing {len(cities)} cities...")

## 2. Refined Selection Step (Sequential Stationarity)

We evaluate each city. A city is selected if it has high correlation AND can be made stationary through our standard pipeline.

In [ ]:
def get_stationary_transform(series, seasonal_period=7):
    """Returns (step_name, transform_func) or (None, None)."""
    def is_stationary(s):
        try:
            return adfuller(s.dropna())[1] < 0.05
        except:
            return False

    if is_stationary(series):
        return "Raw", lambda s: s
    
    try:
        s_log = np.log(series)
        if is_stationary(s_log): return "Log", lambda s: np.log(s)
        
        s_diff = s_log.diff()
        if is_stationary(s_diff): return "Log+Diff", lambda s: np.log(s).diff()
        
        s_seasonal = s_diff.diff(seasonal_period)
        if is_stationary(s_seasonal): 
            return "Log+Diff+Seasonal", lambda s: np.log(s).diff().diff(seasonal_period)
    except:
        pass
        
    return None, None

## 3. Batch Evaluation (Adapting select_best_controls loop)

In [ ]:
results = []
df_pre = df.loc[pre_beg:pre_end]

for target in tqdm(cities, desc="Targets"):
    for city in cities:
        if target == city:
            continue
            
        corr_raw = df_pre[target].corr(df_pre[city])
        step_name, transform_func = get_stationary_transform(df_pre[city])
        
        corr_trans = None
        granger_p = None
        var_ratio = None
        
        if step_name:
            s_city_trans = transform_func(df_pre[city]).dropna()
            s_target_trans = transform_func(df_pre[target]).dropna()
            
            # Behavioral Correlation
            joined = pd.concat([s_city_trans, s_target_trans], axis=1).dropna()
            corr_trans = joined.iloc[:, 0].corr(joined.iloc[:, 1])
            
            # Granger Causality
            try:
                granger_result = grangercausalitytests(joined[[target, city]], maxlag=2, verbose=False)
                granger_p = granger_result[1][0]['params_ftest'][1]
            except: granger_p = 1.0
            
            # Variance Ratio (Volatility Matching)
            var_ratio = s_city_trans.std() / s_target_trans.std()
        
        # Initial Assignment with Variance and Correlation Filter
        tier = "None"
        if corr_trans and corr_trans > t1_thresh:
            if 0.5 < var_ratio < 2.0:
                tier = "Tier 1 (Strict)"
            else:
                tier = "Rejected (High Variance)"
        elif corr_trans and corr_trans > t2_thresh and corr_raw > t2_thresh:
            tier = "Tier 2 (Fallback)"
        
        results.append({
            'Target': target,
            'City': city,
            'Correlation_Raw': corr_raw,
            'Correlation_Transformed': corr_trans,
            'Granger_p_value': granger_p,
            'Variance_Ratio': var_ratio,
            'Selection_Tier': tier
        })

pairings_df = pd.DataFrame(results)
print(f"\nPrepared {len(pairings_df)} pairings for further analysis.")

## 4. Power and Volume Calculator

We estimate the **Minimum Detectable Effect (MDE)** to ensure the experiment is sufficiently powered. These metrics are added to all tiered pairings and exported to CSV.

In [ ]:
def calculate_mde_refined(target_series, control_series, post_days=14, confidence=0.95, power=0.8):
    import statsmodels.api as sm
    from scipy.stats import norm
    
    # 1. Critical values
    z_alpha = norm.ppf(1 - (1 - confidence) / 2)
    z_beta = norm.ppf(power)
    z_combined = z_alpha + z_beta
    
    # 2. Get high-frequency residual noise (Stationary Noise)
    y = target_series.diff().dropna()
    x = control_series.diff().dropna()
    
    x_reg = sm.add_constant(x)
    model = sm.OLS(y, x_reg).fit()
    sigma_daily = np.sqrt(model.mse_resid)
    
    # 3. Detection Threshold for the TOTAL sum over post_days
    total_incremental_units_required = z_combined * sigma_daily * np.sqrt(post_days)
    
    # Percentage MDE vs baseline total volume
    baseline_total_volume = target_series.mean() * post_days
    mde_pct = total_incremental_units_required / baseline_total_volume
    
    return mde_pct, total_incremental_units_required

def augment_with_power(df_pairs, df_all, durations=[14, 30]):
    """Calculates power metrics for all entries with a valid selection tier."""
    augmented_rows = []
    # We only calculate power for Tier 1 and Tier 2 to save time
    valid_pairs = df_pairs[df_pairs['Selection_Tier'].str.contains("Tier")].copy()
    
    for idx, row in tqdm(valid_pairs.iterrows(), total=len(valid_pairs), desc="Calculating Power"):
        target, city = row['Target'], row['City']
        s_target = df_all[target].loc[pre_beg:pre_end]
        s_city = df_all[city].loc[pre_beg:pre_end]
        
        res = row.to_dict()
        for d in durations:
            mde_p, mde_a = calculate_mde_refined(s_target, s_city, post_days=d)
            res[f"Volume_{d}d"] = s_target.mean() * d
            res[f"MDE_{d}d (%)"] = mde_p
            res[f"Impact_{d}d_Total"] = mde_a
        augmented_rows.append(res)
    
    return pd.DataFrame(augmented_rows)

# Run Power Analysis
augmented_pairings = augment_with_power(pairings_df, df)

# Merge back with non-tiered results just for the CSV export completeness (optional, but requested)
final_export_df = pd.concat([augmented_pairings, pairings_df[~pairings_df['Selection_Tier'].str.contains("Tier")]], sort=False)
final_export_df.to_csv('city_pairings.csv', index=False)
print(f"\nSaved augmented data to city_pairings.csv")

# Preview the Top Rows for the notebook display
display_df = augmented_pairings[augmented_pairings['Selection_Tier'] == 'Tier 1 (Strict)'].sort_values('Correlation_Transformed', ascending=False).head(10)

# Formatting for Jupyter Display
cols_v = [c for c in display_df.columns if "Volume" in c]
cols_pct = [c for c in display_df.columns if "(%)" in c]
cols_abs = [c for c in display_df.columns if "Total" in c]
display_cols = ["Target", "City"] + cols_v + cols_pct + cols_abs

format_dict = {col: "{:.2%}" for col in cols_pct}
format_dict.update({col: "{:.0f}" for col in cols_abs})
format_dict.update({col: "{:.0f}" for col in cols_v})

display_df[display_cols].style.format(format_dict)

### Glossary & Interpretation Guide

| Term | Definition |
| :--- | :--- |
| **Target** | The city where you plan to launch the campaign/intervention. |
| **Volume_{d}d** | The total expected sales units in the target city over a {d}-day period. |
| **Control** | The city acting as the benchmark. |
| **MDE (%)** | **Minimum Detectable Effect**. The smallest percentage lift in total sales required over the period for statistical significance. |
| **Impact_{d}d_Total** | **Absolute Detection Threshold**. The total number of *additional* units you need to sell across the **entire {d}-day window** to be sure the result is real. |

### Example: Target = Barcelona | Control = City_18

| Target | Control | Volume_14d | MDE_14d (%) | Impact_14d_Total | Volume_30d | MDE_30d (%) | Impact_30d_Total |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Barcelona** | **City_18** | **35,000** | **12.85%** | **4,500** | **75,000** | **8.80%** | **6,600** |

**How to read this:**

1. **The 14-Day Scenario (2 Weeks)**
- **Volume_14d (35,000)**: Over 14 days, Barcelona normally sells 35,000 units total.
- **Impact_14d_Total (4,500)**: To be statistically significant, your campaign must generate **4,500 extra units** on top of the 35k.
- **MDE_14d (12.85%)**: This means you need a **+12.85% lift** in total sales to "break through" the daily market noise.

2. **The 30-Day Scenario (1 Month)**
- **Volume_30d (75,000)**: Over 30 days, sales grow to 75,000 units.
- **Impact_30d_Total (6,600)**: Running the test longer improves sensitivity. You only need **6,600 extra units** (not 9,000!) to reach significance.
- **MDE_30d (8.80%)**: Your required lift dropped from ~13% to **under 9%**. The test is much more powerful if you wait longer.